# MATH 441 Optimization Problems

### Outline for Today
* The Production problem (Resource Allocation)
* The Consumption problem (Resource Extraction)
* Examples in groups

### Production (aka Resource Allocation)

#### Problem Statement

A production facility uses $m$ types of raw materials (like iron, oil, basic elements, ...) to produce $n$ types of products (like steel, plastic, fertilizer, ...). The amount of each material is limited. How much of each product should the facility produce to maximize profit?




#### Define Variables

* $\rho_i$ is the cost per unit of material $i=1,\dots,m$
* $\sigma_j$ is the price per unit of product $j=1,\dots,n$
* $a_{ij}$ is the amount of material $i$ require to produce a unit of product $j$
* $b_i$ is the amount of material $i$ available
* $x_j$ is the number of units of product $j$ to make (decision variables)
* $c_j$ is the overall profit per unit of product $j$, so its revenue minus its cost



#### Identify Constraints and Make Assumptions

* The amount of each product produced by the facility does not change the market price
* The amount of each material is limited

#### Build Solutions

The profit per unit of product $j$ is revenue minus cost:

$$
c_j = \sigma_j - \sum_i \rho_i a_{ij}
$$

The total profit to maximize is

$$
\mathbf{c}^T \mathbf{x} = \sum_j c_j x_j = \sum_j \sigma_j x_j  - \sum_j \sum_i \rho_i a_{ij} x_j
$$

subject to the contraint

$$
\sum_j a_{ij} x_j \leq b_i \ , \ \ i = 1,\dots,m
$$

$$ x_j > 0, \ \ j = 1,\dots, m$$

This is a linear programming problem that we can solve using the simplex method or any LP solver: maximize $\mathbf{c}^T \mathbf{x}$ subject to $A \mathbf{x} \leq \mathbf{b}$.

See Linear Programming (Vanderbei) Section 1.1.


In [44]:
import numpy as np
import cvxpy as cp
import pandas as pd

In [45]:
m=3 # Number of materials
n=4 # Number of products

In [46]:
sigma =np.array([10,7,15,10]) # Revenue of each product
A = np.array([[2.  , 1.  , 3.  , 2.  ],
              [1.  , 1.75, 1.  , 1.  ],
              [2.  , 3.5 , 5.  , 2.  ]]) # Amount of material for each product
rho = np.array([0.90 , 0.80 , 0.24]) # Cost per unit of material
b = np.array([100,110,200]) # How much of each material is available

Let's solve this problem using cvxpy

In [47]:
X = cp.Variable(n)

In [48]:
obj = cp.Maximize(sigma @ X -(A @ X) @ rho)
constraint_0 = [A@X<=b]
constraint_1 = [X>=0]
prob = cp.Problem(obj,constraint_0+constraint_1)

In [ ]:
prob.solve()

In [ ]:
X.value.round(3)

### Consumption (aka Resource Extraction)

#### Problem Statement

I would like make sure that I meet my recommended dietary allowance each day - enough calories, carbs, fats, proteins, etc.. Based on the food available to me at the supermarket, how can I do this while minimizing the cost?

For example, one slice of pizza costs $5 and gives 200 calories, 30g carbs, 10g fat, 10g protein etc.



#### Define Variables

* $c_j$ 
* $x_j$ 
* $a_{ij}$ 
* $b_i$ 



#### Identify Constraints and Make Assumptions

* I need to have sufficient amounts of each nutrient.
* I can have as many of each food item as needed.

#### Build Solutions

The total shopping cost

$$
\mathbf{c}^T \mathbf{x} = \sum_j c_j x_j
$$

The mineral requirement constraints are

$$
\sum_j a_{ij} x_j \ ?? \ b_i \ , \ \ i = 1,\dots,m
$$

$$x_j >0, \ \ j = 1,\dots,n$$

This is also a linear programming problem that we can solve!

See Linear Programming (Vanderbei) Problem 5.16 p. 86.

In [51]:
n=4 # Number of food items
m=3 # Number of nutrients

In [52]:
c =np.array([10,7,15,10]) # Cost of each food item
A = np.array([[2.  , 1.  , 3.  , 2.  ],
              [1.  , 1.75, 1.  , 1.  ],
              [2.  , 3.5 , 5.  , 2.  ]]) # Amount of each nutrient in each food item
b = np.array([100,110,200]) # How much of each nutrient I need

In [53]:
X = cp.Variable(n)

In [54]:
obj = cp.Minimize(c @ X)
constraint_0 = [A@X>=b]
constraint_1 = [X>=0]
prob = cp.Problem(obj,constraint_0+constraint_1)


In [ ]:
prob.solve()

In [ ]:
X.value.round(3)

#### Let's use real data and see what results we get.

Loading in the real data:

In [57]:
food = pd.read_csv("food.csv")

In [58]:
c = np.array(food['Cost'])
A = np.array(food[['Protein','Carbohydrate','Fat']]).T
b = np.array([200,350,150]) # These are our requirements

In [59]:
X = cp.Variable(food.shape[0])

In [60]:
obj = cp.Minimize(c @ X)
constraint_0 = [A@X>=b]
constraint_1 = [X>=0]
prob = cp.Problem(obj,constraint_0+constraint_1)

In [ ]:
prob.solve()

In [ ]:
for i in range(food.shape[0]):
    print(food.iloc[i]['Name'],abs(X.value.round(3)[i]))

#### What other optimization problems fit either a resource allocation problem or a resource extraction problem?

##### Extra: Comparison between these two problems

Resource Allocation:

$$
\text{max } \mathbf{c}^T \mathbf{x} \text{ with the constraint } A\mathbf{x}\leq b
$$

Resource Extraction:

$$
\text{min } \mathbf{c}^T \mathbf{x} \text{ with the constraint } A\mathbf{x}\geq b
$$

Are these the same problem but with different values?

We know that max $\mathbf{c}^T \mathbf{x}$ = - min $-\mathbf{c}^T \mathbf{x}$ 

But are what about the constraints?
$$Ax\leq b \implies -Ax\geq -b$$




This means we can rewrite the resource allocation problem as an extraction problem

$$
\text{max } \mathbf{c}^T \mathbf{x} \text{ with the constraint } A\mathbf{x}\leq b
$$

$$
\implies -\text{min } -\mathbf{c}^T \mathbf{x} \text{ with the constraint } A\mathbf{x}\leq b
$$

$$
\implies -\text{min } -\mathbf{c}^T \mathbf{x} \text{ with the constraint } -A\mathbf{x}\geq -b
$$

Let's test it!

In [63]:
n=4 # number of food items
m=3 # number of nutrients
c =-np.array([ 6.92,  3.86, 10.3 ,  6.92]) # cost of each food item
A = -np.array([[2.  , 1.  , 3.  , 2.  ],
              [1.  , 1.75, 1.  , 1.  ],
              [2.  , 3.5 , 5.  , 2.  ]]) # Amount of each nutrient in each food item
b = -np.array([100,110,200]) # how much of each nutrient I need

In [64]:
X = cp.Variable(n)

In [65]:
obj = cp.Minimize(c @ X)
constraint = [A@X>=b, X>=0]
prob = -cp.Problem(obj,constraint)


In [ ]:
prob.solve()


In [ ]:
X.value.round(3)